In [1]:
from gurobipy import *

In [2]:
#Parameters
try:
    plant, capacity_limit = multidict({
        ('Macon'): 18000,
        ('Louisville'): 15000,
        ('Detroit'): 25000,
        ('Phoenix'): 20000})
    
    distributor, demand = multidict({
        ('Tacoma'): 8500,
        ('San Diego'): 14500,
        ('Dallas'): 13500,
        ('Denver'): 12600,
        ('St. Louis'): 18000,
        ('Tampa'): 15000,
        ('Baltimore'): 9000})

    production_cost = {
        ('Macon','Tacoma'): 35.5,
        ('Macon','San Diego'): 35.5,
        ('Macon','Dallas'): 35.5,
        ('Macon','Denver'): 35.5,
        ('Macon','St. Louis'): 35.5,
        ('Macon','Tampa'): 35.5,
        ('Macon','Baltimore'): 35.5,
        ('Louisville','Tacoma'): 37.5,
        ('Louisville','San Diego'): 37.5,
        ('Louisville','Dallas'): 37.5,
        ('Louisville','Denver'): 37.5,
        ('Louisville','St. Louis'): 37.5,
        ('Louisville','Tampa'): 37.5,
        ('Louisville','Baltimore'): 37.5,
        ('Detroit','Tacoma'): 39.0,
        ('Detroit','San Diego'): 39.0,
        ('Detroit','Dallas'): 39.0,
        ('Detroit','Denver'): 39.0,
        ('Detroit','St. Louis'): 39.0,
        ('Detroit','Tampa'): 39.0,
        ('Detroit','Baltimore'): 39.0,
        ('Phoenix','Tacoma'): 36.25,
        ('Phoenix','San Diego'): 36.25,
        ('Phoenix','Dallas'): 36.25,
        ('Phoenix','Denver'): 36.25,
        ('Phoenix','St. Louis'): 36.25,
        ('Phoenix','Tampa'): 36.25,
        ('Phoenix','Baltimore'): 36.25}
    
finally:   
    shipping_cost = {
        ('Macon','Tacoma'): 2.5,
        ('Macon','San Diego'): 2.75,
        ('Macon','Dallas'): 1.75,
        ('Macon','Denver'): 2.00,
        ('Macon','St. Louis'): 2.10,
        ('Macon','Tampa'): 1.80,
        ('Macon','Baltimore'): 1.65,
        ('Louisville','Tacoma'): 1.85,
        ('Louisville','San Diego'): 1.90,
        ('Louisville','Dallas'): 1.50,
        ('Louisville','Denver'): 1.60,
        ('Louisville','St. Louis'): 1.00,
        ('Louisville','Tampa'): 1.90,
        ('Louisville','Baltimore'): 1.85,
        ('Detroit','Tacoma'): 2.30,
        ('Detroit','San Diego'): 2.25,
        ('Detroit','Dallas'): 1.85,
        ('Detroit','Denver'): 1.25,
        ('Detroit','St. Louis'): 1.50,
        ('Detroit','Tampa'): 2.25,
        ('Detroit','Baltimore'): 2.00,
        ('Phoenix','Tacoma'): 1.90,
        ('Phoenix','San Diego'): 0.90,
        ('Phoenix','Dallas'): 1.60,
        ('Phoenix','Denver'): 1.75,
        ('Phoenix','St. Louis'): 2.00,
        ('Phoenix','Tampa'): 2.50,
        ('Phoenix','Baltimore'): 2.65}

In [7]:
## Model Generation
m = Model('Transportation')

## Create variables 
x = m.addVars(plant, distributor, lb=0, name='Sentry Lock Corporation')

## Constarints
m.addConstrs( (((quicksum(x[i,j] for i in plant)) <= demand[j]) for j in distributor), name ='demand_limit')
m.addConstrs( (((quicksum(x[i,j] for i in plant)) >= 0.8*demand[j]) for j in distributor), name ='demand_80_limit')
m.addConstrs( (((quicksum(x[i,j] for j in distributor)) == capacity_limit[i]) for i in plant), name ='capacity_limit')
m.addConstrs(x[i, j] >= 0 for i , j in x)

# Objective function
obj = quicksum(shipping_cost[i, j]*x[i,j] for i in plant for j in distributor) + quicksum(production_cost[i,j]*x[i,j] for i in plant for j in distributor)

In [8]:
# Solving
m.setObjective(obj, GRB.MINIMIZE)
m.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 46 rows, 28 columns and 112 nonzeros
Model fingerprint: 0xe3cd280e
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+01, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [7e+03, 3e+04]
Presolve removed 35 rows and 0 columns
Presolve time: 0.03s
Presolved: 11 rows, 35 columns, 63 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.886000e+04   0.000000e+00      0s
      13    3.0113600e+06   0.000000e+00   0.000000e+00      0s

Solved in 13 iterations and 0.05 seconds (0.00 work units)
Optimal objective  3.011360000e+06


In [9]:
# Results
m.printAttr('x')       #values of variables
m.printAttr('Pi')      #shadow prices
m.printAttr('RC')      #Reduced costs


    Variable            x 
-------------------------
Sentry Lock Corporation[Macon,Tampa]        12000 
Sentry Lock Corporation[Macon,Baltimore]         6000 
Sentry Lock Corporation[Louisville,Tacoma]          600 
Sentry Lock Corporation[Louisville,St. Louis]        14400 
Sentry Lock Corporation[Detroit,Tacoma]          400 
Sentry Lock Corporation[Detroit,Dallas]        10800 
Sentry Lock Corporation[Detroit,Denver]        12600 
Sentry Lock Corporation[Detroit,Baltimore]         1200 
Sentry Lock Corporation[Phoenix,Tacoma]         5800 
Sentry Lock Corporation[Phoenix,San Diego]        14200 

  Constraint           Pi 
-------------------------
demand_limit[Denver]        -0.05 
demand_80_limit[Tacoma]            1 
demand_80_limit[Dallas]         0.55 
demand_80_limit[St. Louis]         0.15 
demand_80_limit[Tampa]         0.85 
demand_80_limit[Baltimore]          0.7 
capacity_limit[Macon]        36.45 
capacity_limit[Louisville]        38.35 
capacity_limit[Detroit]         